In [ ]:
import requests
import json
from chembl_webresource_client.new_client import new_client

In [ ]:
query_payload = '''{
  "size": 24,
  "from": 0,
  "_source": [
    "target_chembl_id"
  ],
  "query": {
    "bool": {
      "must": [
        {
          "query_string": {
            "analyze_wildcard": true,
            "query": "_metadata.related_compounds.all_chembl_ids:(\\"CHEMBL866\\")"
          }
        }
      ],
      "filter": []
    }
  },
  "track_total_hits": true,
  "sort": []
}
'''
headers = {'Content-Type': 'application/json'}

In [ ]:
#This function modifies the json payload above, sets the queried chembl id to the passed in 
# @param chebml. We cannot search for targets related to a compound through the "chembl_webresource_client"
# package as our query involves metadata on compounds which is not accessible through that package.
def chemblQuery(chembl):
    payload = json.loads(query_payload)
    payload['query']['bool']['must'][0]['query_string']['query'] = f'_metadata.related_compounds.all_chembl_ids:("{chembl}")'
    
    #Run the target search query against CHEMBL's public API. Sends the header
    r = requests.post("https://www.ebi.ac.uk/chembl/elk/es/chembl_target/_search",headers=headers,data=json.dumps(payload))
    
    #Extracts the CHEMBL ids from the query we sent.
    #print(r.json())
    hits = r.json()['hits']['hits']
    
    #For every CHEMBL from our query, isolate just the CHEMBL id, return these chembl ids as a list.
    hits = [x['_source']['target_chembl_id'] for x in hits]

    return hits

In [ ]:
chembl_idxs = chemblQuery("CHEMBL866")
for chembl_idx in chembl_idxs:
    print(f"CHEMBL IDX: {chembl_idx}")

In [ ]:
for chembl_idx in chembl_idxs:
    target = new_client.target.filter(target_chembl_id=chembl_idx)
    print(f"Response from chembl_webresource_client on {chembl_idx}")
    print(target)